In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder

<h3>Загрузка и предобработка датасета</h3>

In [2]:
#загрузка датасета
df = pd.read_csv('coords_data_numerated.csv')
df = df.drop(columns=['Unnamed: 0', 'GSR'])
df.head()

,id,BreathingType,TimeStamp_sec,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
0,1,0,0.058895,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0,0.111889,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0,0.158371,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0,0.262516,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0,0.295620,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


In [3]:
#Определение выходного признака и входных признаков
y = df['BreathingType']
X = df.drop(columns=['BreathingType'])
X.tail()

,id,TimeStamp_sec,FirstMarkerXCoord,FirstMarkerYCoord,FirstMarkerZCoord,SecondMarkerXCoord,SecondMarkerYCoord,SecondMarkerZCoord,ThirdMarkerXCoord,ThirdMarkerYCoord,ThirdMarkerZCoord
230921,258,59.76925,1.063,-1.649,3.843,1.093,-1.839,3.624,1.030,-1.867,3.859
230922,258,59.80188,1.062,-1.648,3.842,1.094,-1.841,3.625,1.030,-1.867,3.857
230923,258,59.90696,1.062,-1.648,3.842,1.093,-1.841,3.624,1.030,-1.868,3.855
230924,258,59.92954,1.063,-1.647,3.842,1.094,-1.841,3.624,1.029,-1.868,3.853
230925,258,59.99477,1.063,-1.647,3.842,1.094,-1.841,3.623,1.030,-1.868,3.853


In [4]:
#переименование столбцов
dict_renames = {
    'FirstMarkerXCoord' : 'FMX',
    'FirstMarkerYCoord' : 'FMY',
    'FirstMarkerZCoord' : 'FMZ',
    'SecondMarkerXCoord' :  'SMX',
    'SecondMarkerYCoord' : 'SMY',
    'SecondMarkerZCoord' : 'SMZ',
    'ThirdMarkerXCoord' : 'TMX',
    'ThirdMarkerYCoord' : 'TMY',
    'ThirdMarkerZCoord' : 'TMZ'
}
X = X.rename(columns=dict_renames)
X.head()

,id,TimeStamp_sec,FMX,FMY,FMZ,SMX,SMY,SMZ,TMX,TMY,TMZ
0,1,0.058895,0.689,-1.453,4.417,0.881,-1.544,4.547,0.718,-1.607,4.369
1,1,0.111889,0.687,-1.452,4.418,0.881,-1.544,4.547,0.715,-1.607,4.371
2,1,0.158371,0.687,-1.451,4.419,0.881,-1.544,4.545,0.714,-1.607,4.371
3,1,0.262516,0.685,-1.450,4.420,0.882,-1.544,4.541,0.711,-1.608,4.373
4,1,0.295620,0.684,-1.449,4.420,0.883,-1.544,4.540,0.709,-1.607,4.370


<h3>Извлечение признаков с помощью библиотеки Cesium</h3>

In [5]:
import cesium

In [6]:
from cesium import featurize

In [13]:
from cesium import data_management

In [7]:
df_parsed_features = pd.read_csv('features_parsed.csv')
df_parsed_features

,match,group,is_participating,start,end,content
0,1,1,yes,3655,3679,all_times_nhist_numpeaks
1,2,1,yes,3809,3834,all_times_nhist_peak1_bin
2,3,1,yes,4305,4330,all_times_nhist_peak2_bin
3,4,1,yes,4800,4825,all_times_nhist_peak3_bin
4,5,1,yes,5296,5321,all_times_nhist_peak4_bin
...,...,...,...,...,...,...
111,112,1,yes,26299,26316,p2p_scatter_2praw
112,113,1,yes,26482,26502,p2p_scatter_over_mad
113,114,1,yes,26618,26644,p2p_scatter_pfold_over_mad
114,115,1,yes,26796,26818,p2p_ssqr_diff_over_var


In [8]:
#расчет для всех 9 столбцов
numeric_cols = X.drop(columns=['id', 'TimeStamp_sec'])

In [9]:
features_to_use = df_parsed_features['content'].to_list()
features_to_use

['all_times_nhist_numpeaks',
 'all_times_nhist_peak1_bin',
 'all_times_nhist_peak2_bin',
 'all_times_nhist_peak3_bin',
 'all_times_nhist_peak4_bin',
 'all_times_nhist_peak_1_to_2',
 'all_times_nhist_peak_1_to_3',
 'all_times_nhist_peak_1_to_4',
 'all_times_nhist_peak_2_to_3',
 'all_times_nhist_peak_2_to_4',
 'all_times_nhist_peak_3_to_4',
 'all_times_nhist_peak_val',
 'avg_double_to_single_step',
 'avg_err',
 'avgt',
 'cad_probs_1',
 'cad_probs_10',
 'cad_probs_20',
 'cad_probs_30',
 'cad_probs_40',
 'cad_probs_50',
 'cad_probs_100',
 'cad_probs_500',
 'cad_probs_1000',
 'cad_probs_5000',
 'cad_probs_10000',
 'cad_probs_50000',
 'cad_probs_100000',
 'cad_probs_500000',
 'cad_probs_1000000',
 'cad_probs_5000000',
 'cad_probs_10000000',
 'cads_avg',
 'cads_kurtosis',
 'cads_med',
 'cads_skew',
 'cads_std',
 'mean',
 'med_double_to_single_step',
 'med_err',
 'n_epochs',
 'std_double_to_single_step',
 'std_err',
 'total_time',
 'amplitude',
 'anderson_darling',
 'flux_percentile_ratio_mid2

In [14]:
def calc_cesium_features(prefix, col_values, time_col):
    ts = data_management.TimeSeries(t=time_col.to_list(), m=col_values.to_list())
    features_extracted = featurize.featurize_single_ts(
        ts,
        features_to_use=features_to_use,
    )
    features_extracted = features_extracted.dropna()
    features_extracted = features_extracted.reset_index()
    features_extracted = features_extracted.drop(columns=['channel'])
    col_features = pd.DataFrame()
    col_features = col_features.reindex(columns= features_extracted['feature'].to_list())
    col_features.loc[0] = features_extracted[0].to_list()
    for column in col_features.columns:
        col_features.rename(columns={column : prefix+column}, inplace=True)
    return col_features

In [16]:
#расчет новых признаков
new_features = pd.DataFrame()
for i in range(1, 259):
    print(i)
    ts_features = pd.DataFrame()
    #расчет статистических показателей по каждому столбцу
    for col in numeric_cols:
        col_values = X[X['id'] == i] [col]
        time_col = X[X['id'] == i]['TimeStamp_sec']
        prefix = col+"_"
        cesium_features = calc_cesium_features(prefix, col_values, time_col)
        ts_features = pd.concat([ts_features, cesium_features], axis=1)
    new_features = pd.concat([new_features,ts_features], axis=0, ignore_index=True)

1
2
3
4
5


C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: invalid value encountered in true_divide
  slopes = np.diff(x) / np.diff(t)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in true_divide
  out_dict["folded_slopes"] = np.diff(y_2per_fold) / np.diff(t_2per_fold)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: invalid value encountered in true_divide
  slopes = np.diff(x) / np.diff(t)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in true_divide
  out_dict["folded_slopes"] = np.diff(y_2per_fold) / np.diff(t_2per_fold)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: divide by zero encountered in 

6
7


C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: divide by zero encountered in true_divide
  slopes = np.diff(x) / np.diff(t)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in true_divide
  out_dict["folded_slopes"] = np.diff(y_2per_fold) / np.diff(t_2per_fold)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: divide by zero encountered in true_divide
  slopes = np.diff(x) / np.diff(t)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in true_divide
  out_dict["folded_slopes"] = np.diff(y_2per_fold) / np.diff(t_2per_fold)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: divide by zero encountered i

8
9


...

256


C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\cadence_features.py:19: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\cadence_features.py:19: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1] - cads[:-2])
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\cadence_features.py:19: RuntimeWarning: divide by zero encountered in true_divide
  return (cads[2:] + cads[:-2]) / (cads[1:-1]

257


C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in true_divide
  out_dict["folded_slopes"] = np.diff(y_2per_fold) / np.diff(t_2per_fold)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: invalid value encountered in true_divide
  slopes = np.diff(x) / np.diff(t)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in true_divide
  out_dict["folded_slopes"] = np.diff(y_2per_fold) / np.diff(t_2per_fold)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\common_functions.py:7: RuntimeWarning: divide by zero encountered in true_divide
  slopes = np.diff(x) / np.diff(t)
C:\Users\Student\AppData\Roaming\Python\Python39\site-packages\cesium\features\period_folding.py:71: RuntimeWarning: invalid value encountered in t

258


In [20]:
new_features = new_features.dropna(axis=1)
new_features.head()

,FMX_all_times_nhist_numpeaks,FMX_all_times_nhist_peak_val,FMX_avg_err,FMX_avgt,FMX_cad_probs_1,FMX_cad_probs_10,FMX_cad_probs_20,FMX_cad_probs_30,FMX_cad_probs_40,FMX_cad_probs_50,...,TMZ_freq_frequency_ratio_31,TMZ_freq_model_max_delta_mags,TMZ_freq_model_min_delta_mags,TMZ_freq_model_phi1_phi2,TMZ_freq_n_alias,TMZ_freq_varrat,TMZ_freq_y_offset,TMZ_linear_trend,TMZ_medperc90_2p_p,TMZ_scatter_res_raw
0,1.0,0.032362,0.0001,30.049381,0.000000,0.000000,0.010000,0.036667,0.117778,0.210000,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
1,1.0,0.032358,0.0001,30.018374,0.000000,0.000000,0.006667,0.016667,0.031111,0.093333,...,6.510204,5.171413e-03,1.712282e-03,0.161412,0.0,0.000002,-0.000077,-0.000094,0.875438,0.647715
2,1.0,0.032284,0.0001,30.020163,0.000000,0.000000,0.000000,0.007778,0.062222,0.172222,...,1.489451,3.630511e-10,2.307628e-11,0.351319,0.0,0.000003,0.000048,0.000041,0.828281,0.488921
3,1.0,0.032308,0.0001,30.044976,0.000000,0.000000,0.008889,0.032222,0.110000,0.203333,...,28.200000,9.721408e-02,8.020429e-02,0.039808,0.0,0.000002,-0.000709,-0.002106,1.247486,0.463907
4,1.0,0.032315,0.0001,30.010141,0.001111,0.001111,0.001111,0.001111,0.001111,0.010000,...,37.133333,2.167187e-10,4.792460e-10,0.318662,0.0,0.000002,-0.004451,0.001067,1.084122,0.811935


In [21]:
new_features.to_csv('cesium_features.csv')

<h3>Добавление новых признаков в датасет</h3>

In [22]:
#загрузка датасета
data = pd.read_csv('breathes.csv')
data = data.drop(columns=['Unnamed: 0'])
data.head()

,ID,Date_Of_Birth,Sex,Had_Covid,Begin_Of_Covid,End_Of_Covid,Lung_Damage,Damage_Percent,Breathing_Type,Frequency,...,Sin_Period23,Sin_Amp23/Sin_Amp13,Sin_Amp12/Sin_Amp13,Sin_Amp12/Sin_Amp23,Sin_Omega23/Sin_Omega13,Sin_Omega12/Sin_Omega13,Sin_Omega12/Sin_Omega23,(Sin_Offset23/Sin_Offset13)^2,(Sin_Offset12/Sin_Offset13)^2,(Sin_Offset12/Sin_Offset23)^2
0,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.178712,0.002273,-0.001832,-0.032392,-0.156378,-0.155622,-0.156309,-0.088206,-0.063808,-0.059907
1,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.183415,0.002486,-0.001808,-0.029759,-0.151883,-0.154821,-0.159698,-0.088211,-0.063809,-0.059906
2,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.177844,0.002607,-0.001792,-0.030606,-0.157144,-0.156177,-0.155971,-0.088215,-0.063811,-0.059905
3,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.182131,0.002219,-0.002349,-0.030553,-0.152623,-0.155988,-0.160060,-0.088219,-0.063815,-0.059905
4,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,-0.191648,0.003473,-0.001519,-0.031048,-0.165326,-0.156451,-0.137226,-0.088204,-0.063811,-0.059908


In [23]:
#добавление новых признаков в датасет
i = 0
new_features_cols = new_features.columns
for id in range(1,87):
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
    i += 1
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'брюшное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1
    for col in new_features_cols:
        data.loc[(data['Breathing_Type'] == 'смешанное') & (data['ID'] == id), col] = new_features.loc[i][col]
    i += 1

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

...

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.loc[(data['Breathing_Type'] == 'грудное') & (data['ID'] == id), col]= new_features.loc[i][col]
C:\Users\Student\AppData\Local\Temp\ipykernel_13352\2099544929.py:6: PerformanceWarning: DataFrame is hi

In [24]:
data.head(15)

,ID,Date_Of_Birth,Sex,Had_Covid,Begin_Of_Covid,End_Of_Covid,Lung_Damage,Damage_Percent,Breathing_Type,Frequency,...,TMZ_freq_frequency_ratio_31,TMZ_freq_model_max_delta_mags,TMZ_freq_model_min_delta_mags,TMZ_freq_model_phi1_phi2,TMZ_freq_n_alias,TMZ_freq_varrat,TMZ_freq_y_offset,TMZ_linear_trend,TMZ_medperc90_2p_p,TMZ_scatter_res_raw
0,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
1,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
2,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
3,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
4,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
5,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,грудное,-0.144237,...,0.625866,4.877229e-11,3.942930e-11,0.397148,0.0,0.000002,-0.000028,0.000102,0.978863,0.522527
6,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,брюшное,-0.464100,...,6.510204,5.171413e-03,1.712282e-03,0.161412,0.0,0.000002,-0.000077,-0.000094,0.875438,0.647715
7,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,брюшное,-0.464100,...,6.510204,5.171413e-03,1.712282e-03,0.161412,0.0,0.000002,-0.000077,-0.000094,0.875438,0.647715
8,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,брюшное,-0.464100,...,6.510204,5.171413e-03,1.712282e-03,0.161412,0.0,0.000002,-0.000077,-0.000094,0.875438,0.647715
9,1,04.07.2003,M,No,00.00.0000,00.00.0000,No,-0.207005,брюшное,-0.464100,...,6.510204,5.171413e-03,1.712282e-03,0.161412,0.0,0.000002,-0.000077,-0.000094,0.875438,0.647715


<h3>Подготовка данных для тестирования признаков</h3>

In [40]:
#отбрасывание ненужных столбцов
categorical_cols = ['Sex', 'Had_Covid', 'Lung_Damage', 'DominatorFreq']
drop_columns = ['Date_Of_Birth', 'Begin_Of_Covid', 'End_Of_Covid'] + categorical_cols
df = data.drop(columns=drop_columns)
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)
#целевой признак
y = df['Breathing_Type']
#определение численных признаков
non_numerical_cols = ['ID', 'Breathing_Type']
X = df.drop(columns=non_numerical_cols)
#трансформация категориальных признаков
le = LabelEncoder()
y = le.fit_transform(y)

In [42]:
#разделение на обучающую(70%) и тестовую (30%) выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [43]:
from sklearn.metrics import recall_score, accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
#расчет метрик по тесовой выборке
def calculate_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Accuracy: ", accuracy)
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-score: ", f1)
    report = classification_report(y_test, y_pred)
    print(report)

#расчет знамости признаков
def calc_feature_importances(model, X_df):
    importance = model.feature_importances_
    feature_importance = pd.DataFrame({'Feature': X_df.columns, 'Importance': importance})
    feature_importance = feature_importance.sort_values('Importance', ascending=False)
    return feature_importance

<h2>Тестирование новых признаков</h2>

<h3>Отбор признаков методом взаимной информации</h3>

In [44]:
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif

selector = GenericUnivariateSelect(mutual_info_classif, mode='k_best', param=50)

In [45]:
X_new = selector.fit_transform(X, y)

In [52]:
importance = selector.scores_
feature_importance = pd.DataFrame({'Feature': X.columns, 'Scores': importance, 'Support': selector.get_support()})
#feature_importance = feature_importance.sort_values('Scores', ascending=False)
feature_importance

,Feature,Scores,Support
0,Damage_Percent,0.002995,False
1,Frequency,0.117928,False
2,Amplitude,1.074615,False
3,DominatorFreqPower,0.008057,False
4,D_1_2_Avg,0.088523,False
...,...,...,...
841,TMZ_freq_varrat,1.056190,False
842,TMZ_freq_y_offset,1.080887,False
843,TMZ_linear_trend,1.076577,False
844,TMZ_medperc90_2p_p,1.094183,True


In [53]:
bad_features = feature_importance[feature_importance['Support'] == False]
drop_columns = list(bad_features['Feature'])
drop_columns
X2 = X.drop(columns=drop_columns)
X2.head()

,FMX_cads_kurtosis,FMX_shapiro_wilk,FMX_stetson_j,FMX_freq2_amplitude1,FMX_freq2_amplitude2,FMX_freq2_rel_phase4,FMX_freq_signif_ratio_21,FMY_cads_kurtosis,FMY_mean,FMY_period_fast,...,TMY_period_fast,TMY_qso_log_chi2_qsonu,TMY_freq1_rel_phase2,TMY_freq2_freq,TMY_freq3_rel_phase3,TMZ_cads_kurtosis,TMZ_freq1_freq,TMZ_freq2_rel_phase3,TMZ_freq_model_phi1_phi2,TMZ_medperc90_2p_p
0,-0.585876,0.981035,-0.993514,0.002028,0.00089,2.052539,0.678722,-0.585876,-1.444629,3.487519,...,3.472975,-2.111355,-0.870182,0.02335,1.233669,-0.585876,0.288872,1.231535,0.397148,0.978863
1,-0.585876,0.981035,-0.993514,0.002028,0.00089,2.052539,0.678722,-0.585876,-1.444629,3.487519,...,3.472975,-2.111355,-0.870182,0.02335,1.233669,-0.585876,0.288872,1.231535,0.397148,0.978863
2,-0.585876,0.981035,-0.993514,0.002028,0.00089,2.052539,0.678722,-0.585876,-1.444629,3.487519,...,3.472975,-2.111355,-0.870182,0.02335,1.233669,-0.585876,0.288872,1.231535,0.397148,0.978863
3,-0.585876,0.981035,-0.993514,0.002028,0.00089,2.052539,0.678722,-0.585876,-1.444629,3.487519,...,3.472975,-2.111355,-0.870182,0.02335,1.233669,-0.585876,0.288872,1.231535,0.397148,0.978863
4,-0.585876,0.981035,-0.993514,0.002028,0.00089,2.052539,0.678722,-0.585876,-1.444629,3.487519,...,3.472975,-2.111355,-0.870182,0.02335,1.233669,-0.585876,0.288872,1.231535,0.397148,0.978863


In [51]:
feature_importance.to_excel('mutual_features.xlsx')

In [56]:
#разделение на обучающую(70%) и тестовую (30%) выборки
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3, random_state=42)

<h3>Полносвязная нейронная сеть</h3>

In [74]:
from keras.models import Sequential
from keras.layers import Dense

In [71]:
#X = [[np.random.randint(1, 100) for i in range(0, 100)], [np.random.randint(1, 100) for i in range(0, 100)], [np.random.randint(1, 100) for i in range(0, 100)], [np.random.randint(1, 100) for i in range(0, 100)], [np.random.randint(1, 100) for i in range(0, 100)]]
#y = [np.random.randint(3) for i in range(0, 100)]

In [99]:
model = Sequential()
model.add(Dense(64, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [100]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=40, batch_size=4)

In [104]:
loss, accuracy = model.evaluate(X_test, y_test)
accuracy

14/14 [==============================] - 0s 0s/step - loss: 0.0494 - accuracy: 0.9905


0.9905437231063843

In [111]:
weights = model.get_weights()
importance = np.mean(np.abs(weights[0]), axis=1)
feature_importance = pd.DataFrame({'Feature': X2.columns, 'Scores': importance*100})
feature_importance = feature_importance.sort_values('Scores', ascending=False)
feature_importance

#for i, importance in enumerate(feature_importances):
#    print(f'Feature {i+1}: {importance}')

,Feature,Scores
11,FMY_std,36.282124
32,SMZ_freq3_rel_phase3,34.712498
37,TMX_freq_model_phi1_phi2,33.749901
15,FMY_freq3_amplitude1,32.104855
4,FMX_freq2_amplitude2,30.066055
42,TMY_freq1_rel_phase2,29.260105
46,TMZ_freq1_freq,29.105324
48,TMZ_freq_model_phi1_phi2,27.480835
20,FMZ_freq1_amplitude4,27.465385
22,FMZ_freq2_rel_phase2,26.381987


In [112]:
feature_importance.to_excel('neural_network_features.xlsx')

<h3>CatBoost</h3>

In [54]:
import catboost as cb

In [55]:
model_cb = cb.CatBoostClassifier()

In [57]:
model_cb.fit(X_train, y_train)

Learning rate set to 0.079066
0:	learn: 1.0727122	total: 169ms	remaining: 2m 49s
1:	learn: 1.0408051	total: 183ms	remaining: 1m 31s
2:	learn: 1.0114156	total: 198ms	remaining: 1m 5s
3:	learn: 0.9797711	total: 213ms	remaining: 53.1s
4:	learn: 0.9503417	total: 226ms	remaining: 45s
5:	learn: 0.9256674	total: 239ms	remaining: 39.6s
6:	learn: 0.9044440	total: 253ms	remaining: 35.8s
7:	learn: 0.8805494	total: 266ms	remaining: 33s
8:	learn: 0.8583530	total: 280ms	remaining: 30.8s
9:	learn: 0.8410210	total: 293ms	remaining: 29s
10:	learn: 0.8241147	total: 306ms	remaining: 27.6s
11:	learn: 0.8026868	total: 319ms	remaining: 26.3s
12:	learn: 0.7822011	total: 333ms	remaining: 25.3s
13:	learn: 0.7651301	total: 346ms	remaining: 24.4s
14:	learn: 0.7469162	total: 359ms	remaining: 23.6s
15:	learn: 0.7312153	total: 372ms	remaining: 22.9s
16:	learn: 0.7199039	total: 387ms	remaining: 22.4s
17:	learn: 0.7047390	total: 402ms	remaining: 22s
18:	learn: 0.6931800	total: 417ms	remaining: 21.6s
19:	learn: 0.6813

170:	learn: 0.0979558	total: 2.59s	remaining: 12.6s
171:	learn: 0.0972880	total: 2.6s	remaining: 12.5s
172:	learn: 0.0967583	total: 2.62s	remaining: 12.5s
173:	learn: 0.0959059	total: 2.63s	remaining: 12.5s
174:	learn: 0.0955712	total: 2.64s	remaining: 12.5s
175:	learn: 0.0946510	total: 2.66s	remaining: 12.4s
176:	learn: 0.0939537	total: 2.67s	remaining: 12.4s
177:	learn: 0.0930974	total: 2.69s	remaining: 12.4s
178:	learn: 0.0924285	total: 2.71s	remaining: 12.4s
179:	learn: 0.0915944	total: 2.72s	remaining: 12.4s
180:	learn: 0.0912568	total: 2.74s	remaining: 12.4s
181:	learn: 0.0903877	total: 2.76s	remaining: 12.4s
182:	learn: 0.0894167	total: 2.78s	remaining: 12.4s
183:	learn: 0.0892880	total: 2.79s	remaining: 12.4s
184:	learn: 0.0882335	total: 2.81s	remaining: 12.4s
185:	learn: 0.0875732	total: 2.82s	remaining: 12.3s
186:	learn: 0.0873148	total: 2.83s	remaining: 12.3s
187:	learn: 0.0869019	total: 2.85s	remaining: 12.3s
188:	learn: 0.0863446	total: 2.86s	remaining: 12.3s
189:	learn: 0

334:	learn: 0.0372142	total: 4.97s	remaining: 9.86s
335:	learn: 0.0371662	total: 4.98s	remaining: 9.84s
336:	learn: 0.0370010	total: 4.99s	remaining: 9.82s
337:	learn: 0.0367962	total: 5.01s	remaining: 9.8s
338:	learn: 0.0365996	total: 5.02s	remaining: 9.79s
339:	learn: 0.0364736	total: 5.03s	remaining: 9.77s
340:	learn: 0.0363038	total: 5.04s	remaining: 9.75s
341:	learn: 0.0361586	total: 5.06s	remaining: 9.73s
342:	learn: 0.0361100	total: 5.07s	remaining: 9.71s
343:	learn: 0.0359985	total: 5.08s	remaining: 9.69s
344:	learn: 0.0358664	total: 5.09s	remaining: 9.67s
345:	learn: 0.0358114	total: 5.11s	remaining: 9.65s
346:	learn: 0.0356528	total: 5.12s	remaining: 9.63s
347:	learn: 0.0354681	total: 5.13s	remaining: 9.61s
348:	learn: 0.0352785	total: 5.14s	remaining: 9.59s
349:	learn: 0.0352570	total: 5.16s	remaining: 9.57s
350:	learn: 0.0350600	total: 5.17s	remaining: 9.56s
351:	learn: 0.0348970	total: 5.18s	remaining: 9.54s
352:	learn: 0.0347674	total: 5.2s	remaining: 9.52s
353:	learn: 0.

504:	learn: 0.0204018	total: 7.15s	remaining: 7s
505:	learn: 0.0203064	total: 7.16s	remaining: 6.99s
506:	learn: 0.0201981	total: 7.17s	remaining: 6.97s
507:	learn: 0.0201350	total: 7.19s	remaining: 6.96s
508:	learn: 0.0200877	total: 7.2s	remaining: 6.94s
509:	learn: 0.0200365	total: 7.21s	remaining: 6.93s
510:	learn: 0.0199742	total: 7.22s	remaining: 6.91s
511:	learn: 0.0199337	total: 7.24s	remaining: 6.9s
512:	learn: 0.0198710	total: 7.25s	remaining: 6.88s
513:	learn: 0.0198248	total: 7.26s	remaining: 6.87s
514:	learn: 0.0197559	total: 7.27s	remaining: 6.85s
515:	learn: 0.0197118	total: 7.29s	remaining: 6.83s
516:	learn: 0.0196839	total: 7.3s	remaining: 6.82s
517:	learn: 0.0196758	total: 7.31s	remaining: 6.8s
518:	learn: 0.0196082	total: 7.33s	remaining: 6.79s
519:	learn: 0.0195742	total: 7.34s	remaining: 6.77s
520:	learn: 0.0194845	total: 7.35s	remaining: 6.76s
521:	learn: 0.0194485	total: 7.37s	remaining: 6.75s
522:	learn: 0.0193694	total: 7.38s	remaining: 6.73s
523:	learn: 0.01927

674:	learn: 0.0132964	total: 9.35s	remaining: 4.5s
675:	learn: 0.0132676	total: 9.36s	remaining: 4.49s
676:	learn: 0.0132348	total: 9.38s	remaining: 4.47s
677:	learn: 0.0131934	total: 9.39s	remaining: 4.46s
678:	learn: 0.0131823	total: 9.41s	remaining: 4.45s
679:	learn: 0.0131484	total: 9.43s	remaining: 4.43s
680:	learn: 0.0131230	total: 9.44s	remaining: 4.42s
681:	learn: 0.0131133	total: 9.45s	remaining: 4.41s
682:	learn: 0.0130720	total: 9.46s	remaining: 4.39s
683:	learn: 0.0130279	total: 9.47s	remaining: 4.38s
684:	learn: 0.0129804	total: 9.49s	remaining: 4.36s
685:	learn: 0.0129401	total: 9.5s	remaining: 4.35s
686:	learn: 0.0129239	total: 9.51s	remaining: 4.33s
687:	learn: 0.0128971	total: 9.53s	remaining: 4.32s
688:	learn: 0.0128767	total: 9.54s	remaining: 4.3s
689:	learn: 0.0128557	total: 9.55s	remaining: 4.29s
690:	learn: 0.0128394	total: 9.56s	remaining: 4.28s
691:	learn: 0.0128135	total: 9.58s	remaining: 4.26s
692:	learn: 0.0128029	total: 9.59s	remaining: 4.25s
693:	learn: 0.0

838:	learn: 0.0097886	total: 11.5s	remaining: 2.21s
839:	learn: 0.0097606	total: 11.6s	remaining: 2.2s
840:	learn: 0.0097368	total: 11.6s	remaining: 2.19s
841:	learn: 0.0097266	total: 11.6s	remaining: 2.17s
842:	learn: 0.0097089	total: 11.6s	remaining: 2.16s
843:	learn: 0.0097053	total: 11.6s	remaining: 2.15s
844:	learn: 0.0097010	total: 11.6s	remaining: 2.13s
845:	learn: 0.0096751	total: 11.6s	remaining: 2.12s
846:	learn: 0.0096474	total: 11.6s	remaining: 2.1s
847:	learn: 0.0096329	total: 11.7s	remaining: 2.09s
848:	learn: 0.0096011	total: 11.7s	remaining: 2.08s
849:	learn: 0.0095946	total: 11.7s	remaining: 2.06s
850:	learn: 0.0095822	total: 11.7s	remaining: 2.05s
851:	learn: 0.0095618	total: 11.7s	remaining: 2.04s
852:	learn: 0.0095375	total: 11.7s	remaining: 2.02s
853:	learn: 0.0095298	total: 11.8s	remaining: 2.01s
854:	learn: 0.0095137	total: 11.8s	remaining: 2s
855:	learn: 0.0094999	total: 11.8s	remaining: 1.98s
856:	learn: 0.0094831	total: 11.8s	remaining: 1.97s
857:	learn: 0.009

In [58]:
y_pred = model_cb.predict(X_test)
calculate_metrics(y_test, y_pred)

Accuracy:  0.9929078014184397
Recall:  0.9929078014184397
Precision:  0.9930459611310676
F1-score:  0.992906619056909
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       149
           1       1.00      1.00      1.00       123
           2       0.98      1.00      0.99       151

    accuracy                           0.99       423
   macro avg       0.99      0.99      0.99       423
weighted avg       0.99      0.99      0.99       423



In [59]:
importance = model_cb.get_feature_importance()

feature_importance = pd.DataFrame({'Feature': X2.columns, 'Importance': importance})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

feature_importance

,Feature,Importance
9,FMY_period_fast,5.785456
37,TMX_freq_model_phi1_phi2,5.422647
10,FMY_qso_log_chi2_qsonu,4.018762
46,TMZ_freq1_freq,3.551287
41,TMY_qso_log_chi2_qsonu,3.321822
15,FMY_freq3_amplitude1,3.164684
32,SMZ_freq3_rel_phase3,3.159426
48,TMZ_freq_model_phi1_phi2,3.110319
11,FMY_std,2.848780
30,SMY_freq_amplitude_ratio_31,2.716032


In [60]:
feature_importance.to_excel('catboost_features.xlsx')